## Safe execution of parallel tasks (advanced)

There are certain situations you need to avoid when running GRASS tools in parallel:

 * write output maps/files with identical names
 * modify computational region when running within the same mapset
 * modify MASK when running within the same mapset
 * modify vector attribute database within the same mapset
 * use [r.reclass](https://grass.osgeo.org/grass-stable/manuals/r.reclass.html) to reclassify from the same base map


The following examples show how to deal with some of these situations when running parallel tasks in the same mapset is desired.

### Safely modifying computational region in a single mapset

Sometimes modifying computational region in a script is needed. It is a good practice to not change the global computational region, which effectively modifies a file in a mapset,
but only change the environment variable `GRASS_REGION`.
Here, we modified the previous viewshed example to compute in parallel viewsheds with different extents:

In [ ]:
%%writefile example.py
import os
from multiprocessing import Pool
import grass.script as gs

def viewshed(point):
    x, y, cat = point
    # copy current environment, modify and pass it to r.viewshed
    env = os.environ.copy()
    maxdistance = 10000
    # create region based on the viewpoint and maxdistance
    env["GRASS_REGION"] = gs.region_env(e=x + maxdistance, w=x - maxdistance, n=y + maxdistance, s=y - maxdistance, align="DEM")
    gs.run_command("r.viewshed", input="DEM", output=f"viewshed_{cat}", coordinates=(x, y), max_distance=maxdistance, env=env)
    return f"viewshed_{cat}"

if __name__ == "__main__":
    viewpoints = [(594231, 275545, 1),
                  (659805, 259566, 2),
                  (646109, 232901, 3),
                  (602946, 203226, 4)]
    with Pool(processes=2) as pool:
        maps = pool.map(viewshed, viewpoints)
    print(maps)

In [ ]:
%run example.py

### Safely modifying vectors with attributes in a single mapset

By default vector maps share a single SQLite database file, however SQLite does not support concurrent write access. That poses a problem when modifying vectors with attributes in parallel. While this can be solved by running the computations in separate mapsets, it is also possible to change the default behavior to write attributes of each vector to the vector's individual SQLite file. This behavior can be activated after a new mapset is created with:

```
 db.connect driver=sqlite database='$GISDBASE/$LOCATION_NAME/$MAPSET/vector/$MAP/sqlite.db'
```

Alternatively, a PostgreSQL or another database backend can be used for attributes to offload the parallel writing to the database system.